In [1]:
import os
import numpy as np
import torch
import glob
from torchvision.transforms import transforms
from torchvision import datasets
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import shutil
import copy
import torch.nn as nn
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
## this will cell will distribute the images into the paths
curr_path = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq"

train_path = curr_path + "/train_set"
train_real = train_path + "/real"
train_fake = train_path + "/fake"

test_path = curr_path + "/test_set"
test_real = test_path + "/real"
test_fake = test_path + "/fake"

val_path = curr_path + "/validate_set"
val_real = val_path + "/real"
val_fake = val_path + "/fake"

if not os.path.exists(train_path):
    os.mkdir(train_path)
    os.mkdir(train_real)
    os.mkdir(train_fake)

if not os.path.exists(test_path):
    os.mkdir(test_path)
    os.mkdir(test_real)
    os.mkdir(test_fake)
if not os.path.exists(val_path):
    os.mkdir(val_path)
    os.mkdir(val_real)
    os.mkdir(val_fake)


real_dir = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq/Real"
fake_dir = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq/Fake"

## distributes into files by a 8:1:1 ration

# count = 0
# for real_im in os.listdir(real_dir):
#     if count < 800:
#         shutil.move(real_dir + "/" + real_im, train_real)
#     elif count > 800 and count < 901:
#         shutil.move(real_dir + "/" + real_im, test_real)
#     else:
#         shutil.move(real_dir + "/" + real_im, val_real)
#     count += 1

# count = 0
# for fake_im in os.listdir(fake_dir):
#     if count < 800:
#         shutil.move(fake_dir + "/" + fake_im, train_fake)
#     elif count > 800 and count < 901:
#         shutil.move(fake_dir + "/" + fake_im, test_fake)
#     else:
#         shutil.move(fake_dir + "/" + fake_im, val_fake)
#     count += 1



In [3]:
## Transforms
transformer = transforms.Compose([
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.25, 1.0), ratio=(0.8, 1.2)),
    transforms.Resize(size=(64, 64), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=(256, 256), scale=(0.25, 1.0), ratio=(0.8, 1.2)),
        transforms.Resize(size=(64, 64), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(size=(256, 256), scale=(0.25, 1.0), ratio=(0.8, 1.2)),
        transforms.Resize(size=(64, 64), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
## DataLoader

train_path = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq/train_set"
test_path = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq/test_set"
val_path = "/Users/nicholask/Desktop/Masters/research/Global_Texture_Enhancement_for_Fake_Face_Detection_in_the-Wild/stylegan-ffhq/validate_set"

train_loader = DataLoader(
    datasets.ImageFolder(train_path, transform=data_transforms["train"]),
    batch_size = 30,
    shuffle=True,
    num_workers = 4
)

test_loader = DataLoader(
    datasets.ImageFolder(test_path, transform=transformer),
    batch_size = 4,
    shuffle=True,
    num_workers = 4
)

val_loader = DataLoader(
    datasets.ImageFolder(val_path, transform=data_transforms["val"]),
    batch_size = 30,
    shuffle=True,
    num_workers = 4
)

dataloaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": len(datasets.ImageFolder(train_path, transform=data_transforms["train"])), "val": len(datasets.ImageFolder(val_path, transform=data_transforms["val"])) }


In [5]:
## Categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split("/")[-1] for j in root.iterdir()])
print(classes)

['fake', 'real']


In [6]:
### From Pytoch tutorials: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
## There are other .pth files found on the github such as stylegan-ffhq.pth that we can load
model_ft = torch.load("graminit.pth")

## Just incase we need to map onto the cpu
# model_ft = torch.load("graminit.pth", map_location='cpu')

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/Users/nicholask/opt/anaconda3/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nicholask/opt/anaconda3/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nicholask/opt/anaconda3/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original s

In [8]:
## Execution time on Mac M1 cpu is 39 minutes
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.4951 Acc: 0.7919
val Loss: 0.7979 Acc: 0.5622

Epoch 1/24
----------
train Loss: 0.2882 Acc: 0.8862
val Loss: 0.2935 Acc: 0.8856

Epoch 2/24
----------
train Loss: 0.2330 Acc: 0.9119
val Loss: 0.2131 Acc: 0.9154

Epoch 3/24
----------
train Loss: 0.2163 Acc: 0.9150
val Loss: 0.1671 Acc: 0.9502

Epoch 4/24
----------
train Loss: 0.1937 Acc: 0.9263
val Loss: 0.1384 Acc: 0.9602

Epoch 5/24
----------
train Loss: 0.1609 Acc: 0.9406
val Loss: 0.1667 Acc: 0.9403

Epoch 6/24
----------
train Loss: 0.1662 Acc: 0.9337
val Loss: 0.2578 Acc: 0.9005

Epoch 7/24
----------
train Loss: 0.1498 Acc: 0.9425
val Loss: 0.1546 Acc: 0.9254

Epoch 8/24
----------
train Loss: 0.1313 Acc: 0.9519
val Loss: 0.1813 Acc: 0.9154

Epoch 9/24
----------
train Loss: 0.1460 Acc: 0.9381
val Loss: 0.1381 Acc: 0.9403

Epoch 10/24
----------
train Loss: 0.1328 Acc: 0.9494
val Loss: 0.1904 Acc: 0.9254

Epoch 11/24
----------
train Loss: 0.1401 Acc: 0.9431
val Loss: 0.1658 Acc: 0.9303

Ep

In [17]:
## testing 

model_ft.eval()

# keep track of the loss and accuracy
test_loss = 0.0
test_accuracy = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        test_accuracy += correct

# calculate average loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = test_accuracy / len(test_loader.dataset)

print('Test Loss: {:.4f}, Test Accuracy: {:.4f}'.format(test_loss, test_accuracy))

Test Loss: 0.3532, Test Accuracy: 0.8543
